# MVKM-ED: Advanced Multi-View Clustering Demo

This notebook demonstrates the advanced capabilities of our MVKM-ED implementation, including:
- GPU acceleration
- Advanced visualizations
- Performance metrics
- Data preprocessing
- Model persistence

In [ ]:
import numpy as np
import torch
from mvkm_ed import MVKMED, MVKMEDConfig
from mvkm_ed.utils import MVKMEDVisualizer, MVKMEDMetrics, MVKMEDDataProcessor
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Generate Sample Multi-View Data

In [ ]:
# Generate synthetic multi-view data
np.random.seed(42)

def generate_multiview_data(n_samples=300, view_dims=[10, 15], n_clusters=3):
    views = []
    centers = np.random.randn(n_clusters, max(view_dims))
    labels = np.random.randint(0, n_clusters, n_samples)
    
    for dim in view_dims:
        view_data = centers[labels, :dim] + 0.1 * np.random.randn(n_samples, dim)
        views.append(view_data)
    
    return views, labels

X, true_labels = generate_multiview_data()
print(f"Generated {len(X)} views of shapes: {[x.shape for x in X]}")

## 2. Preprocess Data

In [ ]:
# Preprocess the views
processor = MVKMEDDataProcessor()
X_processed = processor.preprocess_views(X, scale=True, normalize=True)
print("Data preprocessing complete")

## 3. Configure and Initialize Model

In [ ]:
# Configure the model
config = MVKMEDConfig(
    cluster_num=3,
    points_view=len(X),
    alpha=2.0,
    beta=0.1,
    random_state=42,
    device="auto",  # Will use GPU if available
    verbose=True
)

# Initialize model
model = MVKMED(config)

## 4. Fit Model and Visualize Results

In [ ]:
# Fit the model
model.fit(X_processed)

# Create visualizer
viz = MVKMEDVisualizer(model)

# Plot convergence
plt.figure(figsize=(15, 5))
plt.subplot(121)
viz.plot_convergence()

plt.subplot(122)
viz.plot_view_weights()
plt.tight_layout()

## 5. Evaluate Clustering Quality

In [ ]:
# Compute clustering metrics
metrics = MVKMEDMetrics.compute_metrics(X_processed, model.labels_, true_labels)

print("\nClustering Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

## 6. Save and Load Model

In [ ]:
from mvkm_ed.utils import MVKMEDPersistence

# Save model
MVKMEDPersistence.save_model(model, 'mvkm_ed_model.joblib')

# Load model
loaded_model = MVKMEDPersistence.load_model('mvkm_ed_model.joblib')
print("Model successfully saved and loaded")